In [21]:
import logging

logging.basicConfig(level=logging.WARNING)
root_logger = logging.getLogger()
root_logger.propagate = True


# Set constants

In [22]:
ema_list = [7,14]

In [23]:
test_data_path = "./tmp/test.csv"

# Load test data

In [24]:
import yfinance as yf
import pandas as pd
from NNTrade.common.candle_col_name import INDEX, OPEN,CLOSE, HIGH, LOW, VOLUME

In [25]:
import os

if not os.path.exists(test_data_path):
    data_df:pd.DataFrame =  yf.Ticker("EURUSD=X").history(start="2020-01-01",end="2024-01-01", interval="1d")
    data_df.index = data_df.index.date
    data_df.index.name = INDEX
    data_df["High"] = data_df[["Open","High","Low","Close"]].max(axis=1)
    data_df["Low"] = data_df[["Open","High","Low","Close"]].min(axis=1)
    data_df["Volume"] = 10000000
    data_df.rename({"Open":OPEN, "High": HIGH, "Low":LOW, "Close":CLOSE, "Volume":VOLUME},axis=1, inplace=True)
    data_df.to_csv(test_data_path)

In [26]:
data_df:pd.DataFrame = pd.read_csv(test_data_path, index_col=INDEX)
data_df.index = pd.to_datetime(data_df.index)
from NNTrade.indicators.ma import EmaBuilder
for ema_p in ema_list:
    ema_b = EmaBuilder(ema_p)
    data_df[ema_b.get_name(CLOSE)] = ema_b.get_for(data_df[CLOSE])
data_df

,open,high,low,close,volume,Dividends,Stock Splits,EMA(7)[close],EMA(14)[close]
start_date_time,,,,,,,,,
2020-01-01,1.122083,1.122838,1.115947,1.122083,10000000,0.0,0.0,1.122083,1.122083
2020-01-02,1.121894,1.122712,1.116682,1.122083,10000000,0.0,0.0,1.122083,1.122083
2020-01-03,1.117081,1.118068,1.112570,1.117144,10000000,0.0,0.0,1.120848,1.121424
2020-01-06,1.116246,1.120825,1.115810,1.116196,10000000,0.0,0.0,1.119685,1.120727
2020-01-07,1.119583,1.119946,1.113487,1.119799,10000000,0.0,0.0,1.119714,1.120603
...,...,...,...,...,...,...,...,...,...
2023-12-25,1.102657,1.104240,1.099989,1.102657,10000000,0.0,0.0,1.096208,1.092250
2023-12-26,1.102026,1.103997,1.100958,1.102026,10000000,0.0,0.0,1.097662,1.093554
2023-12-27,1.104301,1.112248,1.102925,1.104301,10000000,0.0,0.0,1.099322,1.094987


# Set strategy

In [27]:
from src.simulation.config.strategy_config import StrategyConfig
from src.simulation.realization.pyalgotrade.strategy_wrapper import StrategyWrapper, CustomValuesWrapper
from pyalgotrade.bar import BasicBar
from pyalgotrade.strategy.position import LongPosition, Position, ShortPosition
from src.simulation.models import Deal

class MyStrategy(StrategyWrapper):
    def __init__(self, barFeed,  strategy_config: StrategyConfig):
        super().__init__(barFeed, strategy_config)
        self.__last_deal:Deal = None

        self.__instrument = "main"
        
        self.__signal = float(strategy_config["delta"])
        self.__ema_p = int(strategy_config["ema_p"])
        self.__ema_name = EmaBuilder(self.__ema_p ).get_name(CLOSE)
        
        self.__ema:CustomValuesWrapper = self._get_custom_value_feed(self.__instrument, self.__ema_name)
    
    def _get_cur_equity(self)->float:
        return self.getBroker().getEquity()

    def _onBars(self, bars):

        bar: BasicBar = bars[self.__instrument]
        
        if self.__last_deal is not None:
            self.__last_deal.set_last_price(bar.getPrice())

        open_positions = [pos for pos in self.open_positions if isinstance(pos, Position) and not pos.exitActive()]
               
        if bar.getPrice()/self.__ema[-1] - 1 > self.__signal:
            if len(open_positions)==0:
                self.enterLong(self.__instrument, 1000, True)
            else:
                opened_short_pos = [pos for pos in self.open_positions if isinstance(pos, ShortPosition) and not pos.exitActive()]
                if len(opened_short_pos)>0: 
                    pos = opened_short_pos[0]
                    pos.exitMarket()
                elif len(open_positions) < 2:
                    self.enterLong(self.__instrument, 1000, True)
        elif self.__ema[-1]/bar.getPrice() - 1 > self.__signal:
            if len(open_positions)==0:
                self.enterShort(self.__instrument, 1000, True)
            else:
                opened_long_pos = [pos for pos in self.open_positions if isinstance(pos, LongPosition) and not pos.exitActive()]
                if len(opened_long_pos)>0: 
                    pos = opened_long_pos[0]
                    pos.exitMarket()
                elif len(open_positions) < 2:
                    self.enterShort(self.__instrument, 1000, True)


# Optimizing

In [28]:
from src.optimization_analyzer.optimization_analyzer import OptimizationAnalyzer, OptimizationConfig
from NNTrade.common import TimeFrame

In [29]:
from datetime import date
from src.simulation.config import StrategyId
from src.common.candle_config import CandleConfig
from src.common.candle_data_set_config import CandleDataSetConfig
from src.common.date_period import DatePeriod
from src.optimization.config.strategy_config_sets import StrategyConfigSet
str_id = StrategyId("test","1")
cds_cfg = CandleDataSetConfig.BuildFrom(CandleConfig("EURUSD",TimeFrame.D),TimeFrame.D,"main")
dp_cfg = DatePeriod(date(2020,1,1), date(2024,1,1))
opt_config = OptimizationConfig(
    cds_cfg, 
    dp_cfg,
    StrategyConfigSet({
        "delta":[0,0.0001,0.0005],
        "ema_p":[7,14]}))

In [30]:
from src.optimization.optimizer.realization import ComplexOptimizer
from src.optimization.strategy.realization import GridStrategyFactory
from src.optimization.report_comparer.realization.level_rules import LevelRulesComparer
import src.optimization.report_comparer.realization.level_rules.rules as rule
import src.optimization.report_comparer.realization.level_rules.value_extractors as extractors
from src.simulation import absTradingSimulator
from src.simulation.realization.pyalgotrade import PyalgotradeTradingSimulator
from src.simulation.config import StrategyId
from src.optimization_analyzer.period_splitter import DefaultPeriodSplitter,timedelta
from src.optimization_analyzer.model import OptimizationAnalyzReport
from typing import Dict, Tuple
from pandas import DataFrame
from src.simulation.cache.file_storage import FileStorageCache

fsc = FileStorageCache("tmp/test_cache")

report_comparer = LevelRulesComparer(
    rule.LevelRises(),
    {
        0: rule.UntillWin([
            rule.NewMinYieldIsAbove(0),
            rule.NewDeviationInLimit(10),
            rule.LevelRises()
        ]),
        0.04: rule.UntillWin([
            rule.NewMinYieldIsAbove(0),
            rule.NewDeviationInLimit(6),
            rule.ValueRises(extractors.MinYieldPerYearValueExtractor)
        ]),
        0.07: rule.UntillWin([
            rule.NewMinYieldIsAbove(0),
            rule.NewDeviationInLimit(4),
            rule.ValueRises(extractors.MinYieldPerYearValueExtractor)
        ])
    })

strategy_wrapper_factory = StrategyWrapper.Factory({
    str_id: lambda bf,cfg:MyStrategy(bf,cfg),
})
def get_trading_sim()->absTradingSimulator:
    data_source = {"EURUSD":data_df}
    trading_simulator = PyalgotradeTradingSimulator(data_source, strategy_wrapper_factory,fsc)
    return trading_simulator

def get_opt_analyzer(trading_simulator: absTradingSimulator)->OptimizationAnalyzer:
    grid_strategy_optimization_f = GridStrategyFactory(report_comparer)
    optimizer = ComplexOptimizer(trading_simulator, grid_strategy_optimization_f)
    opt_analyzer = OptimizationAnalyzer(optimizer, period_splitter=DefaultPeriodSplitter.from_days(360*3, 90*3))
    return opt_analyzer

In [31]:
trading_sim = get_trading_sim()
opt_analyzer = get_opt_analyzer(trading_sim)
analyzation_report_arr = opt_analyzer.analis_optimization_flow(str_id, opt_config,False)
anal_rep_df = OptimizationAnalyzReport.flow_to_dataframe(analyzation_report_arr)
anal_rep_df

100%|██████████| 1/1 [00:00<00:00, 11.98it/s]


PROM  \
                                                                                                                  deals   
                                                                                                                forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)             
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                 15.73703   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      0.548454   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_1   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      1.329749   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_2   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      1.512459   

                                                                                                              avg_net_income  \
                                                                                                                       deals   
                                                                                                                     forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                   8050.950463   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                  14135.850037   

                                                                                                                              \
                                                                                                                               
                                      

# Report view

In [32]:
anal_rep_df

PROM  \
                                                                                                                  deals   
                                                                                                                forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)             
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                 15.73703   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      0.548454   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_1   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      1.329749   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_2   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      1.512459   

                                                                                                              avg_net_income  \
                                                                                                                       deals   
                                                                                                                     forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                   8050.950463   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                  14135.850037   

                                                                                                                              \
                                                                                                                               
                                      

In [33]:
from src.simulation.models.simulation_log import SimulationLog
SimulationLog(analyzation_report_arr[0].forward.capital_log,analyzation_report_arr[0].forward.deal_list).to_dict()

{'capital_log': {datetime.datetime(2022, 12, 16, 0, 0): 1000000,
  datetime.datetime(2022, 12, 19, 0, 0): 1000000.0,
  datetime.datetime(2022, 12, 20, 0, 0): 1000002.9549598694,
  datetime.datetime(2022, 12, 21, 0, 0): 1000005.4366588593,
  datetime.datetime(2022, 12, 22, 0, 0): 1000002.053976059,
  datetime.datetime(2022, 12, 23, 0, 0): 1000000.7040500641,
  datetime.datetime(2022, 12, 26, 0, 0): 1000013.5989189148,
  datetime.datetime(2022, 12, 27, 0, 0): 1000008.0823898315,
  datetime.datetime(2022, 12, 28, 0, 0): 1000007.4713230133,
  datetime.datetime(2022, 12, 29, 0, 0): 1000005.8882236481,
  datetime.datetime(2022, 12, 30, 0, 0): 1000012.1886730194,
  datetime.datetime(2023, 1, 2, 0, 0): 1000021.9848155975,
  datetime.datetime(2023, 1, 3, 0, 0): 1000015.5808925629,
  datetime.datetime(2023, 1, 4, 0, 0): 999989.4089698792,
  datetime.datetime(2023, 1, 5, 0, 0): 1000001.3110637665,
  datetime.datetime(2023, 1, 6, 0, 0): 999992.8963184357,
  datetime.datetime(2023, 1, 9, 0, 0): 999

In [34]:
from src.optimization_analyzer.model import OptimizationAnalyzReport

In [35]:
analyzation_report_arr[0].to_series()

optimization_0  period   from                    2020-01-01
                         untill                  2020-12-26
                capital  yield                    -0.000063
                         yield/year               -0.000064
                         max_yield                 0.000077
                         max_fall                 -0.000263
                deals    deal_count                      44
                         success_deal_count              26
                         fail_deal_count                 16
                         avg_net_profit         5311.711044
                         avg_net_income        14135.850037
                         avg_net_loss          -8363.550941
                         PROM                      0.548454
optimization_1  period   from                    2020-12-26
                         untill                  2021-12-21
                capital  yield                     0.000131
                         yield/year     

In [36]:
anal_rep_df = OptimizationAnalyzReport.flow_to_dataframe(analyzation_report_arr)
anal_rep_df

PROM  \
                                                                                                                  deals   
                                                                                                                forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)             
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                 15.73703   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      0.548454   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_1   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      1.329749   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_2   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      1.512459   

                                                                                                              avg_net_income  \
                                                                                                                       deals   
                                                                                                                     forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                   8050.950463   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                  14135.850037   

                                                                                                                              \
                                                                                                                               
                                      

In [37]:
anal_rep_df["yield/year"]

capital  \
                                                                                                                forward   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)             
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                 0.000116   

                                                                                                                              \
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                     -0.000064   

                                                                                                                              \
                                                                                                              optimization_1   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                      0.000133   

                                                                                                                              
                                                                                                              optimization_2  
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                 
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                     -0.000032

In [38]:
anal_rep_df["yield/year"].describe()

capital                                             
        forward optimization_0 optimization_1 optimization_2
count  1.000000       1.000000       1.000000       1.000000
mean   0.000116      -0.000064       0.000133      -0.000032
std         NaN            NaN            NaN            NaN
min    0.000116      -0.000064       0.000133      -0.000032
25%    0.000116      -0.000064       0.000133      -0.000032
50%    0.000116      -0.000064       0.000133      -0.000032
75%    0.000116      -0.000064       0.000133      -0.000032
max    0.000116      -0.000064       0.000133      -0.000032

In [39]:
anal_rep_df[["from","untill"]]

from  \
                                                                                                                      period   
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                    2020-01-01   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_1   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                    2020-12-26   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_2   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                    2021-12-21   

                                                                                                                      untill  \
                                                                                                                      period   
                                                                                                              optimization_0   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                    2020-12-26   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_1   
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                  
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                    2021-12-21   

                                                                                                                              
                                                                                                                              
                                                                                                              optimization_2  
(optimization, period, from) (optimization, period, untill) (forward, period, from) (forward, period, untill)                 
2020-01-01                   2022-12-16                     2022-12-16              2023-09-12                    2022-12-16

In [40]:
anal_rep_df.columns

MultiIndex([(              'PROM',   'deals',        'forward'),
            (              'PROM',   'deals', 'optimization_0'),
            (              'PROM',   'deals', 'optimization_1'),
            (              'PROM',   'deals', 'optimization_2'),
            (    'avg_net_income',   'deals',        'forward'),
            (    'avg_net_income',   'deals', 'optimization_0'),
            (    'avg_net_income',   'deals', 'optimization_1'),
            (    'avg_net_income',   'deals', 'optimization_2'),
            (      'avg_net_loss',   'deals',        'forward'),
            (      'avg_net_loss',   'deals', 'optimization_0'),
            (      'avg_net_loss',   'deals', 'optimization_1'),
            (      'avg_net_loss',   'deals', 'optimization_2'),
            (    'avg_net_profit',   'deals',        'forward'),
            (    'avg_net_profit',   'deals', 'optimization_0'),
            (    'avg_net_profit',   'deals', 'optimization_1'),
            (    'avg_net